In [1]:
import os
import openstudio

osm_name_box = 'TestShoeBox.osm'
current_dir = os.getcwd()
osm_path = os.path.join(current_dir,osm_name_box)
osm_path = openstudio.path(osm_path) # I guess this is how it wants the path for the translator
print(osm_path)

c:/TheBlackHole/CLT Graduate Research/Gabes_OpenStudio_shared_resources/load_model_add_ruleset/TestShoeBox.osm


In [2]:
t = True
if t == True:
    translator = openstudio.osversion.VersionTranslator()
    osm = translator.loadModel(osm_path).get()
else:
    osm = openstudio.model.Model.load(osm_path).get()
# print("Building model \n\n",osm)

In [3]:
# Found this little bit of code here:
# https://unmethours.com/question/2125/anyone-developing-python-code-for-direct-idf-manipulation/
# So I guess I pass the model object INTO openstudio.model in order to engage with it.
# Good grief -_-
zones = [zone for zone in openstudio.model.getThermalZones(osm)]

# Let's try renaming a zone
print(zones[0].name().get())
print(dir(zones[0].name()))
zones[0].setName('Test Rename')
print(zones[0].name().get())
# Ok that works, let's see if I can get the load schedules I'm after

Core Thermal
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__ref__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__swig_destroy__', '__weakref__', 'get', 'isNull', 'is_initialized', 'reset', 'set', 'this', 'thisown']
Test Rename


In [4]:
print(osm.building().get().name())

print('\n Schedule names')
schedules = osm.getSchedules()
for sch in schedules:
    print(sch.name())

print('\n Electric Equipment Definitions')
loads = osm.getElectricEquipmentDefinitions()
for l in loads:
    print(l.name())

Building 1

 Schedule names
Small Office Bldg Occ
Small Office Activity
Office Bldg Light
Small Office Bldg Equip
Small Office Infil Quarter On

 Electric Equipment Definitions
189.1-2009 - Office - WholeBuilding - Sm Office - CZ4-8 Electric Equipment Definition


In [5]:
osm_model_options = dir(osm)

In [6]:
print('\n Electric Equipment Definitions')
loads = openstudio.model.getElectricEquipmentDefinitions(osm)
# for l in loads:
#     print(l)

added_load = loads[0]
print(added_load)
added_load.setName('New Load')
added_load.setWattsperSpaceFloorArea(0)
added_load.setDesignLevel(100)
print(added_load)
osm.addObject(added_load)
loads = openstudio.model.getElectricEquipmentDefinitions(osm)
for load in loads:
    print(load.name().get())


 Electric Equipment Definitions
OS:ElectricEquipment:Definition,
  {4efb2316-c70e-4c42-be79-37a1bb1fb333}, !- Handle
  189.1-2009 - Office - WholeBuilding - Sm Office - CZ4-8 Electric Equipment Definition, !- Name
  Watts/Area,                             !- Design Level Calculation Method
  ,                                       !- Design Level {W}
  5.81251162502325,                       !- Watts per Space Floor Area {W/m2}
  ;                                       !- Watts per Person {W/person}


OS:ElectricEquipment:Definition,
  {4efb2316-c70e-4c42-be79-37a1bb1fb333}, !- Handle
  New Load,                               !- Name
  EquipmentLevel,                         !- Design Level Calculation Method
  100,                                    !- Design Level {W}
  ,                                       !- Watts per Space Floor Area {W/m2}
  ;                                       !- Watts per Person {W/person}


New Load
Electric Equipment Definition 1


In [7]:
# Let's try making an object from scratch and then adding it to the model
def electric_load_template(os_model, name, design_level):
    # function to take in the info and returns an OS load object
    load = openstudio.model.ElectricEquipmentDefinition(os_model)
    load.setName(name)
    load.setDesignLevel(design_level)
    return load

new_load = electric_load_template(osm, "A Test Load", 490)
print(new_load)

current_loads = print([load.name().get() for load in openstudio.model.getElectricEquipmentDefinitions(osm)])

# Ok this is pretty neat. The line load = openstudio.model.ElectricEquipmentDefinition(os_model)
# makes a new object in the model that I can then edit. I don't even have to save it! It 
# just makes it!

OS:ElectricEquipment:Definition,
  {42c62e72-6a4a-49a4-92ca-38fb17c39a7b}, !- Handle
  A Test Load,                            !- Name
  EquipmentLevel,                         !- Design Level Calculation Method
  490,                                    !- Design Level {W}
  ,                                       !- Watts per Space Floor Area {W/m2}
  ;                                       !- Watts per Person {W/person}


['New Load', 'Electric Equipment Definition 1', 'A Test Load']


In [8]:
# Let's try to make a new schedule now
# Handy unmethours link related to inserting a schedule from a csv
# https://unmethours.com/question/14348/can-you-load-schedulefile-in-openstudio/
import random
def make_an_interval_schedule(os_model, schedule_values, name):
    """Creates a new interval schedule for an Openstudio Model and returns the object

    Args:
        os_model (Openstudio Object): Desired Openstudio Model
        schedule_values (List object): A list of the values for the schedule
        name (String): Name of the schedule

    Returns:
        OS Schedule Object: Returns the newly created schedule object
    """
    # os_vector = openstudio.Vector(len(timeseries_list))
    # for i in timeseries_list:
    #     os_vector[i] = timeseries_list[i]
    # date = openstudio.Date.new(openstudio.MonthOfYear.new("Jan"), 1, 2009)
    # time = openstudio.Time.new(0,1,0,0)
    # timeseries = openstudio.Timeseries.new(date,time,os_vector)
    
    # hard coding the interval until I figure the rest out, then I'll back track
    # so I can infer it from the data given
    interval = openstudio.Time(0,1,0)
    print(interval)
    # Hard cording the units 
    unit_choice = 'unitless'
    start_date = openstudio.Date(openstudio.MonthOfYear(1),1)
    print(start_date)
    time_series = openstudio.TimeSeries(start_date, interval, schedule_values, unit_choice)
    new_schedule = openstudio.model.ScheduleInterval.fromTimeSeries(time_series,os_model)
    new_schedule = new_schedule.get()
    new_schedule.setString(4,'Yes') # This sets the schedule to interpolate based on the timestep, little clunky but I couldn't find the OS object directly
    new_schedule.setName(name)
    
    return new_schedule

# lets make a OS Vector list for every hour in a year with random values representing
# hourly loads.
number_of_timesteps = 8746
schedule_values = openstudio.Vector(number_of_timesteps,0)
for i in range(number_of_timesteps):
    schedule_values[i]= random.random()

# print(schedule_values)
new_schedule = make_an_interval_schedule(osm,schedule_values,'Test Schedule')
schedule_attributes = dir(new_schedule)
# print(new_schedule)

# Ok, We've got the test ruleset schedule, let's assign it to a load in a thermal zone!
# Looks like I assign the equipment itself to a space. Weird but I'll try it.

01:00:00
2009-Jan-01


In [9]:
print('\n Spaces')
spaces = openstudio.model.getSpaces(osm)
first = spaces[0]
print(first)

test_load = openstudio.model.ElectricEquipment(new_load)
test_load.setSpace(first)
test_load.setName('Test Load')
test_load.setSchedule(new_schedule)
print(first.electricEquipment())
# print(dir(new_load))


 Spaces
OS:Space,
  {c1340cb4-3df6-4c36-b10a-cb6e109bd7fa}, !- Handle
  Core,                                   !- Name
  {5626772e-e4b5-4916-a858-ec64414cbf71}, !- Space Type Name
  ,                                       !- Default Construction Set Name
  ,                                       !- Default Schedule Set Name
  ,                                       !- Direction of Relative North {deg}
  ,                                       !- X Origin {m}
  ,                                       !- Y Origin {m}
  ,                                       !- Z Origin {m}
  {ff0cf330-1416-4f59-8826-b943c624951d}, !- Building Story Name
  {ecf3276e-21a7-4401-9779-a93c6ab13047}, !- Thermal Zone Name
  ,                                       !- Part of Total Floor Area
  ,                                       !- Design Specification Outdoor Air Object Name
  ;                                       !- Building Unit Name


(<openstudio.openstudiomodelgeometry.ElectricEquipment; proxy of 

In [10]:
for i in first.electricEquipment():
    print(i.name())
    
# Bonkers, so I've gotta make the load definition, 
# and then get an ElectricEquipment object that I've passed it into, 
# and THEN I can assign it to a space? Guess I better save this 
# and load it in the application and see what I've done.

Test Load


In [11]:
# saving the model
new_name = 'osm_added_load_and_rulset.osm'
osm_save_path = os.path.join(os.getcwd())
openstudio.model.saveModel(osm,new_name,osm_save_path)

True